In [1]:

import sys
import os

google_colab = False
# where directory kits19_tumor is located in your gdrive


base_path = '/content/drive/My Drive/kits'
#here are all scripts and model weights
all_scripts = os.path.join(base_path,'kits19_tumor')
#this is folder with data
data_path = os.path.join(base_path,'kits_cases_subset')



if 'google.colab' in sys.modules:
  google_colab = True

if google_colab:
  print("have google.colab")

  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

  #base_path = os.getcwd()
  sys.path.append(all_scripts)

#for metacentrum
if google_colab == False:
  sys.path.append(os.environ['SCRATCHDIR'] + '/kits19_tumor' )

 

have google.colab
Mounted at /content/drive


In [2]:
from constants import *
%tensorflow_version 1.x
import tensorflow as tf
tf.keras.backend.clear_session()
from tensorflow import keras

import Kits19_methods as k19
import data_load as dl
import generator_train as gt
from model import Unet2D_small,Unet2D_big, Unet3D
from pspNet_small import pspnet_small
from pspNet_big import pspnet_big
from denseUNet_small import Dense_UNet_small
from denseUNet_big import Dense_UNet_big
from deeplabv3plus import Deeplabv3
from keras.regularizers import l1, l2
from keras.layers import Input

from loss_function import make_loss, softmax_dice_loss_tune_params
import time

from keras.optimizers import Adam
from keras.utils import multi_gpu_model, to_categorical
from sklearn.utils import class_weight
from keras import backend as K


from tensorflow.python.client import device_lib

print('keras : ' + str(keras.__version__) )
print('tf : ' + str(tf.__version__) )

TensorFlow 1.x selected.


Using TensorFlow backend.


keras : 2.2.4-tf
tf : 1.15.2


In [3]:

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
from slider import slice_viewer
from keras.models import Sequential,load_model
from generator_train import DataGenerator, MyIterator, MyIterator3D

use_softmax = True
num_classes = 3
use_class_weight = True
lr = 0.0001
regularizer=l2(0.0001)

#rescale = 128 or 512
picture_size = 128
picture_shape = picture_size
num_of_scans = 64 #number of pictures in scans

# Parameters --- just to get whole patint in one read/ generator not used for training
# generator used only for reading data from files, not training
batch_size = 1500 
params_gen = {
          'batch_size':batch_size, 
          'dim':(picture_size,picture_size),
          'rescale':picture_size, 
          'n_channels':3,
          'shuffle':True
         }

In [4]:
def rgb_to_onehot(rgb_arr, color_dict):
    num_classes = len(color_dict)
    shape = rgb_arr.shape[:2]+(num_classes,)
    arr = np.zeros( shape, dtype=np.int8 )
    for i, cls in enumerate(color_dict):
        arr[:,:,i] = np.all(rgb_arr.reshape( (-1,3) ) == color_dict[i], axis=1).reshape(shape[:2])
    return arr

def onehot_to_rgb(onehot, color_dict):
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in color_dict.keys():
        output[single_layer==k] = color_dict[k]
    return np.uint8(output)

def get_class_weights(y_train):
  labels = np.argmax(y_train, axis=-1).flat

  cls_weight = class_weight.compute_class_weight(
      'balanced',
      np.unique(labels),
      labels
    )
  quadratic_class_weight = cls_weight * cls_weight
  return cls_weight

  print('Plain weights', cls_weight)
  print('Quadratic weights', quadratic_class_weight)

color_dict = {0: (255,   0, 0),
              1: (0, 0,   0),
              2: (0, 0,   255)}


In [5]:
#-------------------------------------------------------------#


# Datasets [0,...,209]
#-------------------------------------------------------------#
# change here patient_num 
#-------------------------------------------------------------#
# patient_num --- number of patient in dataset [0,..., 209]
# Datasets [0,...,209]
# Patients [0,1,...199] - training set
# Patients [200,...,209] - testing set
patient_num = 202

IDs_of_files = [patient_num]
# Generators
training_generator = DataGenerator(IDs_of_files, data_path = data_path, **params_gen)
dataTemp = next(training_generator)
x_train = dataTemp[0]
y_train = dataTemp[1] 



In [6]:
#------------------------------------------------------------#
# testing 2D models
# testing 2D models


# !WARNING because of the size of trained models there are weigts only for 2D Unet and 3D Unet!
# !WARNING because of the size of trained models there are weigts only for 2D Unet and 3D Unet!
# !WARNING because of the size of trained models there are weigts only for 2D Unet and 3D Unet!
# additional models can be provided after contacting me on detkotom@fit.cvut.cz

#loading model model weights
model_saved_here = 'output_final_models'
model_weights = '/weights_2D.h5'

#------------------------------------------------------------#
# change folder_name to whatever name of folder with trained model
#------------------------------------------------------------#

dict_folder_model_name = {
    0: ("Unet2D_small_2020_05_11-17_07_37", "Unet2D_small"),
    1: ("pspnet_small_2020_05_23-12_22_54", "pspnet_small"),
    2: ("pspnet_big_2020_05_23-16_04_36", "pspnet_big"),
    3: ("Deeplabv3_2020_05_24-10_23_53", "Deeplabv3"),
    4: ("Dense_UNet_small_2020_05_15-11_38_41", "Dense_UNet_small"),
    5: ("Dense_UNet_big_2020_05_19-19_18_55", "Dense_UNet_big"),
    
}
# possible name of folder
# Unet2D_small_2020_05_11-17_07_37 - BEST
# pspnet_small_2020_05_23-12_22_54
# pspnet_big_2020_05_23-16_04_36
# Deeplabv3_2020_05_24-10_23_53
# Dense_UNet_small_2020_05_15-11_38_41
# Dense_UNet_big_2020_05_19-19_18_55

#------------------------------------------------------------#
# possible model_name to corresponding folder name mentioned below
#------------------------------------------------------------#
# possible model_names -- variable needs to correspond to folder name 
# Unet2D_small 
# pspnet_small 
# pspnet_big 
# Dense_UNet_small 
# Dense_UNet_big 
# Deeplabv3
#------------------------------------------------------------#

# this gives me folder where Unet2D is stored
folder_name, model_name = dict_folder_model_name[0]
regularizer = None

path_to_saved_model = os.path.join(all_scripts, model_saved_here, folder_name)
print(path_to_saved_model)

/content/drive/My Drive/kits/kits19_tumor/output_final_models/Unet2D_small_2020_05_11-17_07_37


In [7]:
K.clear_session()

# for 2D model
# for 2D model
# for 2D model

if model_name == 'Unet2D_small':
  model = Unet2D_small(input_size = (picture_size, picture_size, 3), use_softmax = True, num_classes = num_classes, regularizer=regularizer )

elif model_name == 'pspnet_small':
  model = pspnet_small(num_classes = num_classes, input_shape = (picture_size, picture_size, 3), regularizer = regularizer )

elif model_name == 'pspnet_big':
  model = pspnet_big(num_classes = num_classes, input_shape = (picture_size, picture_size, 3), regularizer = regularizer)

elif model_name == 'Dense_UNet_small':
  input_shape = (picture_shape,picture_shape,3)
  input_img = Input(input_shape, name='model_input')
  model = Dense_UNet_small(input_img,num_classes = num_classes, dropout=0.05, regularizer = regularizer)

elif model_name == 'Dense_UNet_big':
  model = Dense_UNet_big(input_shape = (picture_size, picture_size, 3), num_classes = num_classes, regularizer = regularizer)

elif model_name == 'Deeplabv3':
  model = Deeplabv3( input_shape = (picture_shape,picture_shape,3), num_classes=num_classes, regularizer = regularizer, load_weights_from_internet = False)
else:
  raise ValueError("Network is not implemented yet")

if google_colab:
  if os.path.exists(path_to_saved_model + model_weights):
    model.load_weights( path_to_saved_model+model_weights )
    for i in range(5):
      print('Weights loaded !')

model.compile(optimizer = Adam(lr = lr), loss = 'categorical_crossentropy', metrics=['accuracy','mae'] )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/content/drive/My Drive/kits/kits19_tumor/model.py:84: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv7)


Weights loaded !
Weights loaded !
Weights loaded !
Weights loaded !
Weights loaded !


In [8]:
out = model.predict(x_train)

from keras.utils import to_categorical

max_arg = np.argmax(out, axis=-1)
out_pic = to_categorical(max_arg, num_classes = 3)

out_img = np.zeros(out_pic.shape)
for i in range(len(out_pic)):
  out_img[i] = onehot_to_rgb(out_pic[i],color_dict)
Ynew = out_img

def only_one_channel(Ynew,channel = None):
  cp = Ynew.copy()
  if channel is not None:
    for i in range(3):
      if channel == i :
        continue
      cp[:,:,:,i] = 0
  return cp
  
slice_viewer(only_one_channel(Ynew), step=1 )


interactive(children=(IntSlider(value=191, description='slice_num', max=382), Output(layout=Layout(height='350…

In [9]:
slice_viewer( y_train, step=1 )


interactive(children=(IntSlider(value=191, description='slice_num', max=382), Output(layout=Layout(height='350…

In [10]:
#--------------------------------
# 3D model starts here
#--------------------------------

In [11]:
#-------------------------------------------------------------#


#-------------------------------------------------------------#
# change here patient_num
#-------------------------------------------------------------#
# patient_num --- number of patient in dataset [0,..., 209]
# Datasets [0,...,209]
# Patients [0,1,...199] - training set
# Patients [200,...,209] - testing set
patient_num = 202

IDs_of_files = [patient_num]
# Generators
training_generator = DataGenerator(IDs_of_files,data_path = data_path, **params_gen)
dataTemp = next(training_generator)

#it is called x_train but in this case these are testing data which model never saw before - it is from testing set
x_train = dataTemp[0]
y_train = dataTemp[1] 

In [12]:
batch_size_3D = 2
patch_size_3D = 32

rest = len(x_train) % patch_size_3D
if(rest != 0):
  x_train = x_train[:-rest]
  y_train = y_train[:-rest]
  
x_train_reshaped = x_train.reshape(( int(len(x_train) / patch_size_3D) ,patch_size_3D,128,128,3))
y_train_reshaped = y_train.reshape(( int(len(y_train) / patch_size_3D),patch_size_3D,128,128,3))

print(x_train_reshaped.shape)
print(y_train_reshaped.shape)

(11, 32, 128, 128, 3)
(11, 32, 128, 128, 3)


In [13]:
# loading weights
model_weights = '/weights_3D.h5'
folder_name = 'Unet3D_2020_05_25-18_53_28'
model_saved_here = 'output_final_models'
if google_colab: 
  path_to_saved_model = os.path.join(all_scripts, model_saved_here, folder_name)

model = Unet3D(input_shape=(patch_size_3D,128,128,3), n_filters=48,depth = 3, use_softmax = use_softmax,n_labels=num_classes, regularizer = regularizer)
        # Compile model
model.compile(optimizer = Adam(lr = lr), loss = 'categorical_crossentropy', metrics=['accuracy','mae'] )
if google_colab:
  if os.path.exists(path_to_saved_model + model_weights):
    pass
    model.load_weights( path_to_saved_model+model_weights )
    for i in range(5):
      print('Weights loaded !')

Weights loaded !
Weights loaded !
Weights loaded !
Weights loaded !
Weights loaded !


In [14]:

gen3D = MyIterator3D( len(x_train_reshaped), batch_size = len(x_train_reshaped), shuffle = False, seed = 42 , training = False , x = x_train_reshaped, y = y_train_reshaped)
out_x,out_y = next(gen3D)
print(out_x.shape)


(11, 32, 128, 128, 3)


In [15]:
def predict_whole_patient(out_x,out_y,model):
  len_x = len(out_x)
  add_one = len(out_x) % 2
  x,y = None,None
  for i in range(len_x):
    tmp_x = out_x[i]

    Ynew = model.predict( tmp_x.reshape( (1,) + tmp_x.shape) )
    Ynew = Ynew.reshape((-1,)+ Ynew.shape[-3:])
    tmp_x = tmp_x.reshape((-1,)+ tmp_x.shape[-3:])

    if x is None:
      x = tmp_x
      y = Ynew
    else:
      x = np.concatenate((x,tmp_x))
      y = np.concatenate((y,Ynew))  
  return x,y


In [16]:
from keras.utils import to_categorical

out_whole_patient_x, out_whole_patient_y = predict_whole_patient(out_x,out_y,model)
x_patient = out_whole_patient_x
Ynew = out_whole_patient_y

max_arg = np.argmax(Ynew, axis=-1)
out_pic = to_categorical(max_arg, num_classes = 3)
out_img = np.zeros(out_pic.shape)
for i in range(len(out_pic)):
  out_img[i] = onehot_to_rgb(out_pic[i],color_dict)
Ynew = out_img


In [17]:
# go to slice_num = 150

slice_viewer(Ynew,step=1)


interactive(children=(IntSlider(value=175, description='slice_num', max=351), Output(layout=Layout(height='350…

In [18]:
# go to slice_num = 150


slice_viewer(y_train,step=1)


interactive(children=(IntSlider(value=175, description='slice_num', max=351), Output(layout=Layout(height='350…

In [19]:
# # saving photos of prediction and ground truth

# net2D = True
# patient = patient_num

# for i in range(start,stop+1):
#   num = i
#   name_new   = 'pic_predicted_3D_'+str(i) 
#   name_train = 'pic_ground_truth_3D_'+str(i)
#   if net2D == True:
#     name_new   = 'pic_predicted_2D_'+str(i) 
#     name_train = 'pic_ground_truth_2D_'+str(i)

#   weak_kidney = Ynew[num]
#   whole_kidney = y_train[num]

#   from PIL import Image
#   import matplotlib
#   new_pic = weak_kidney.copy()
#   arr = whole_kidney.copy()

#   #arr /= 255.0

#   new_pic /= 255.0
#   new_pic = np.clip(new_pic, a_min =0.0, a_max = 1.0) 

#   arr = np.clip(arr, a_min =0.0, a_max = 1.0) 

#   if net2D == False:
#     matplotlib.image.imsave('/content/drive/My Drive/kits/bachelor_thesis/bakalarka_fotky_appendix/patient_'+str(patient)+'/predicted_3D/'+name_new+'.png', new_pic)
#     matplotlib.image.imsave('/content/drive/My Drive/kits/bachelor_thesis/bakalarka_fotky_appendix/patient_'+str(patient)+'/true_3D/'+name_train+'.png', arr)
#   else:
#     matplotlib.image.imsave('/content/drive/My Drive/kits/bachelor_thesis/bakalarka_fotky_appendix/patient_'+str(patient)+'/predicted_2D/'+name_new+'.png', new_pic)
#     matplotlib.image.imsave('/content/drive/My Drive/kits/bachelor_thesis/bakalarka_fotky_appendix/patient_'+str(patient)+'/true_2D/'+name_train+'.png', arr)

In [ ]:

# # tensorboard part

# %load_ext tensorboard
# import tensorflow as tf
# import datetime
# import os

# #os.getcwd()
# os.chdir('/content/drive/My Drive/kits/kits19_tumor')
# %tensorboard --logdir='logs/'

# #tensorboard part
# #tensorboard part